<h1><center> Python project</center></h1>
<h2><center>Current car prices and other relevant parameters from bazos.cz</center></h2>
<h3><center>Daniel Brosko, Vojtěch Suchánek</center></h3>

Our goal is to web-scrape advertisements listed on website bazos.cz, which is currently one of the most used websites for selling used cars in Czech republic. It has more than 15 000 car adds daily. On the other hand, it has really poor search options, which pretty much complicates searching for desired car based on your parameters.

We are going to code algorithm, which will scan adds for the current day, pick those, which fulfill our conditions on date and car type and save their links. Then we will go to each link and save the text of the add. Then we will try to analyze the text of the add to find our parameters.

This approach might also allow for longer time period analysis in further steps - we would collect data periodically and investigate the trends in price changes, number of ads for selected car added during particular days, and more. However, since this project should be designed as one-time run, we decided to limit the data to only current date.

In [1]:
import requests
from bs4 import BeautifulSoup
import re
from datetime import date, datetime, timedelta
import time
import numpy as np
import pandas as pd

The commented line below displays the version of packages so they can be used in requirements.txt.

In [2]:
#pip list

By the code in the following chunk, we checked that we are allowed to scrape particular parts of bazos.cz domain.

In [3]:
bots = requests.get('https://auto.bazos.cz/robots.txt')
#print(bots.text)

From the robots page we can see that our actions done in our projects are allowed, since we are not gonna use these search commands.

In the next chunk, we import two .py scripts where we defined the functions search_model, and n_days_search to filter and include only advertisements relevant to our preferences. More comments on the functions are printed few chunks below - where we print the documentation, but also by looking at .py scripts directly in GitHub.

In [4]:
from model_search import search_model
from n_days_search import n_days_search

In [5]:
print("Enter your desired car model here:")
my_search = input()

Enter your desired car model here:
octavia 3


In [6]:
# Now we run the functions we loaded from script, it will take some time, though
soup_list = search_model(my_search)

/Users/danielb/Desktop/IES/Python/Python-project/model_search.py:34: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 34 of the file /Users/danielb/Desktop/IES/Python/Python-project/model_search.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  no_of_adv_html = BeautifulSoup(page.text, 'html')
/Users/danielb/Desktop/IES/Python/Python-project/model_search.py:70: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and

Initial search for your model successfully finished.


In the next chunk, we filter for the ads added today + max 5 days old. If we compared the number for today (2022-08-30) there were 98 at the time, while the number of all ads for the same car-model input was 2022.

In [9]:
print("Enter the number of past days (max. 5) that you want to include in your search here:")
my_days = input()

Enter the number of past days (max. 5) that you want to include in your search here:
4


In [11]:
# Now we specify on how many past days we want to include in our search
my_days = int(my_days)
list_of_offers_url = n_days_search(my_days, soup_list)

The number of found advertisements matching the criteria: 438 .


In [12]:
# Here we print the documentation for our functions we imported earlier.

help(search_model)

help(n_days_search)

Help on function search_model in module model_search:

search_model(user_input: str)
    Function search_model takes string input, hence it has to be in quotes ("").
    The input should be the name of the car model you would like to get results for,
    e.g. "octávia 3" - there should be no problem even when full Czech alphabet is used.
    
    Then the string is stripped of the characters that are not supposed to be in the search input,
    and if there are more than single word in the input, they are connected by '+' (plus) sign,
    since that is the format that bazos.cz use in their URLs.
    
    Then, the prepared string is paste into the common URL format that bazos.cz use.
    
    The very next step is obtaining the number of found advertisements for user's input from the html source code.
    This number of advertisements is used to select the proper length of the adv. tabs list that we will scrape.
    
    The function returns "soup_list" - the list of html codes for each

Finally, we proceed to Data-mining part, where we extract the desired parameters - year of manufacture, year, and price. 

This is probably the most demanding part of the project - we need to extract the relevant data from unformated text. There is no official format of the text, so we tried to find a way how to extract this information from various formats. The results are not the best since sometimes it happen that our code is not able to recognize the unusual format of the parameter. In further steps, probably implementing some ML algorithm could improve the successful recognition significantly.

We save all of those parameters along with the URLs of particular advertisements. We created a class ResultTable that has two methods - "show_results" and "show_best" by which we can display the best recommended ads for our desired car model.

Hence, now we can take a look on potentially most interesting advertisements for us by following the URLs and checking the entire content of several ads instead of looking at "thousands" of them.

In [13]:
# DATA/TEXT MINING PART
def modify_text(text):
    """ Internal function modifying the text of the add to better suit for text mining. """
    return text.replace(" ", "").replace(".", "").replace("xxx", "000").replace("-", "")
def get_numbers_from_text(text):
    """ Internal function for finding all numbers in the text. """
    text = modify_text(text)
    pattern = '[.]?[\d]+[\.]?\d*(?:[eE][-+]?\d+)?'
    list_of_numbers = re.findall(pattern, text)
    return list_of_numbers
def find_years(numbers):
    """ Internal function picking numbers which might be years from all numbers in text. """
    numbers = [x for x in numbers if (float(x) > 1980) and (float(x) < 2023)]
    return numbers
def find_km(numbers):
    """ Internal function picking numbers which might be mileage from all numbers in text. """
    numbers = [x for x in numbers if (float(x) > 3000) and (float(x) < 500000)]
    return numbers
def get_context(text, list_of_tokens, year_dictionary = ['egistr', 'rv', 'RV', 'yrob', 'ýrob', 'prov', 'rok', 'Rok'], km_dictionary = ['km', 'Km', 'KM', 'ilomet', 'ajet', 'ájez', 'achom', 'atoč'], context_span=20):
    """ Internal function looking into surroundings of each year and mileage candidate and searching for parts of words defined in dictionaries. """
    #import re
    context = []
    year = 'No match'
    km = 'No match'
    for token in find_years(list_of_tokens):
        all_occurences_indices = [m.start() for m in re.finditer(token, text)]
        for index in all_occurences_indices:
            left_index = max(index - context_span, 0)
            right_index = min(index + context_span, len(text))
            substring = text[left_index: right_index].strip()
            for s in year_dictionary:
                year_find = [m.start() for m in re.finditer(s, substring)]
                if len(year_find) > 0:
                    year = token
    for token in find_km(list_of_tokens):
        all_occurences_indices = [m.start() for m in re.finditer(token, text)]
        for index in all_occurences_indices:
            left_index = max(index - context_span, 0)
            right_index = min(index + context_span, len(text))
            substring = text[left_index: right_index].strip()
            for s in km_dictionary:
                km_find = [m.start() for m in re.finditer(s, substring)]
                if len(km_find) > 0:
                    km = token
    return [year, km]

class ResultTable(pd.core.frame.DataFrame):
    def show_results(self, min_price = 0, max_price = 10000000, min_year = 1950, max_year = 2022, min_mileage = 0, max_mileage = 500000):
        temp_table = self[(self["mileage"] != "No match") & (self["year_of_manuf"] != "No match")]
        temp_table2 = temp_table[(pd.to_numeric(temp_table["price"]) > min_price) & (pd.to_numeric(temp_table["price"]) < max_price) &
                  (pd.to_numeric(temp_table["year_of_manuf"]) > min_year) & (pd.to_numeric(temp_table["year_of_manuf"]) < max_year) &
                    (pd.to_numeric(temp_table["mileage"]) > min_mileage) & (pd.to_numeric(temp_table["mileage"]) < max_mileage)].sort_values(by = "price")
        print(temp_table2)
    def show_best(self, n = 5, penalty = 5000):
        temp_table = self[(self["mileage"] != "No match") & (self["year_of_manuf"] != "No match")]
        temp_table = temp_table.assign(score=lambda x: ((2022 - pd.to_numeric(x.year_of_manuf))*penalty +
                                       pd.to_numeric(x.mileage)) / pd.to_numeric(x.price))
        temp_table = temp_table.sort_values(by = "score").head(n)
        print(temp_table)
def get_info(links):
    """ Get_info is a final function performing text mining and creating results in form of ResultTable class. """
    results_temp = []
    for i in links:
        print(i)
        add_page = requests.get(i)
        soup_add = BeautifulSoup(add_page.text, 'html')
        add = modify_text(soup_add.find('div', {'class':'popisdetail'}).get_text())
        price = soup_add.find('table').find_all('b')[-1].get_text()
        all_numbers = get_numbers_from_text(add)
        context_got = get_context(add, all_numbers)
        result = [i, context_got[0], context_got[1], price.replace(" ", "").replace("Kč", "")]
        results_temp.append(result)
        time.sleep(0.2)
    results = ResultTable(results_temp)
    results.columns = ['link', 'year_of_manuf', 'mileage', 'price']
    results = results[(results["price"] != "Dohodou") & (results["price"] != "Vtextu") & (results["price"] != "Nabídněte")]
    return results
result = get_info(list_of_offers_url)


pd.options.display.max_colwidth = 120
test = ResultTable(result)
test.show_results(min_price = 50000, max_price = 350000, min_year = 2013, max_year = 2018, min_mileage = 100000, max_mileage = 200000)
test.show_best(n = 10)

https://auto.bazos.cz/inzerat/157680831/skoda-octavia-3-fc-16tdi-85kw-koupcr1majiteltazne2017.php
https://auto.bazos.cz/inzerat/157649412/skoda-octavia-3-20tdi-110kw-dsg-plna-zaruka-2-roky-zdarma.php
https://auto.bazos.cz/inzerat/157495572/skoda-octavia-3-fc-14tsi-81kw-dsg-style-koupcr111tkm2018.php
https://auto.bazos.cz/inzerat/157457320/skoda-octavia-3-rs-20tsi-169kw-dsg-zaruka-2-roky-zdarma.php
https://auto.bazos.cz/inzerat/157452665/octavia-scout-4x420tdi-110kw6stmancolumbuspouze-113tkm.php
https://auto.bazos.cz/inzerat/157425576/skoda-octavia-3-16tdi-85kw-dsg-plna-zaruka-2-roky-zdarma.php
https://auto.bazos.cz/inzerat/157421675/skoda-octavia-3-16tdi-85kw-dsg-plna-zaruka-2-roky-zdarma.php
https://auto.bazos.cz/inzerat/157371003/skoda-octavia-3-16tdi-85kw-dsg-plna-zaruka-2-roky-zdarma.php
https://auto.bazos.cz/inzerat/157333276/skoda-octavia-3-16tdi-85kw-dsg-plna-zaruka-2-roky-zdarma.php
https://auto.bazos.cz/inzerat/157317134/octavia-iii-14-tsi-dsg-business-style-dph.php
https://au

https://auto.bazos.cz/inzerat/157538454/vstrikovaci-lista-04l130089g-20tdi-110kw-skoda-octavia-3.php
https://auto.bazos.cz/inzerat/156914984/skoda-octavia-3-dsg-16tdi-77kw-xenon-stav.php
https://auto.bazos.cz/inzerat/156377700/skoda-octavia-3-karoq-kodiaq-hlava-motoru.php
https://auto.bazos.cz/inzerat/157487262/skoda-octavia-iii-12tsi-77kw-model-2014.php
https://auto.bazos.cz/inzerat/155970833/skoda-octavia-3-20tdi-110kw-dsg-elviko-led-appconnect.php
https://auto.bazos.cz/inzerat/155970730/skoda-octavia-3-20tdi-110kw-canton-dab-nakup-v-klidu.php
https://auto.bazos.cz/inzerat/157489562/vstrikovaci-trysky-04l130277ae-20tdi-135kw-49tis-km.php
https://auto.bazos.cz/inzerat/157489077/prevodovka-5q-manual-start-stop-ptb-12tsi-63kw-cyva-2017.php
https://auto.bazos.cz/inzerat/155900740/skoda-octavia-3-zadni-led-svetla-facelift.php
https://auto.bazos.cz/inzerat/157479736/hybridni-disky-16-octavia-3.php
https://auto.bazos.cz/inzerat/157475039/ruzne-dvere-octavia-3-candy.php
https://auto.bazos.cz

https://auto.bazos.cz/inzerat/157666817/skoda-octavia-superb-ii-18-kola-5x112-nove.php
https://auto.bazos.cz/inzerat/157664411/18-alu-kola-5x112-audi-a3-skoda-octavia3-zimni.php
https://auto.bazos.cz/inzerat/157344356/prodam-skoda-octavia-3-16tdi-81kw.php
https://auto.bazos.cz/inzerat/157662260/skoda-octavia-iii-14-tsi-combi.php
https://auto.bazos.cz/inzerat/157662065/skoda-octavia-3-rs-20tdi-dsg.php
https://auto.bazos.cz/inzerat/157660991/vstriky-20-tdi-110kw.php
https://auto.bazos.cz/inzerat/157660779/skoda-octavia-3-facelift-predni-naraznik-s-maskou.php
https://auto.bazos.cz/inzerat/157660728/skoda-octavia-3-facelift-predni-naraznik.php
https://auto.bazos.cz/inzerat/156464866/skoda-octavia-3-rs-20-tdi-dsg.php
https://auto.bazos.cz/inzerat/157659337/skoda-octavia-3-kombi-16-tdi-top-stav.php
https://auto.bazos.cz/inzerat/157658657/subwoofer-do-skoda-octavia-3-liftback.php
https://auto.bazos.cz/inzerat/157656722/skoda-octavia-iii-rs-20-tdi.php
https://auto.bazos.cz/inzerat/157655724/sk

https://auto.bazos.cz/inzerat/157609301/octavia-3-rs-volantzkosenylikerkakryt-popelnikuruck.php
https://auto.bazos.cz/inzerat/157609176/predam-cierne-klucky-krytky-kluciek-skoda-volkswagen.php
https://auto.bazos.cz/inzerat/157607343/octavia-3.php
https://auto.bazos.cz/inzerat/157606933/prodam-svetlo-skoda-octavia-3-full-led.php
https://auto.bazos.cz/inzerat/157606919/originalni-alu-kola-skoda-se-zimnimi-pneu-20560-r16.php
https://auto.bazos.cz/inzerat/157604632/kryty-pedalu-z-uslechtile-oceli-golf-7-octavia-3-leon3-atp.php
https://auto.bazos.cz/inzerat/157604300/alu-kola-5x112-octavia-3.php
https://auto.bazos.cz/inzerat/157604262/skoda-octavia-3-iii-facelift-predni-naraznik.php
https://auto.bazos.cz/inzerat/157287397/airbag-sada-skoda-octavia-3.php
https://auto.bazos.cz/inzerat/157603518/delici-sit-skoda-octavia-3.php
https://auto.bazos.cz/inzerat/157287073/origo-airbag-sada-skoda-octavia-3.php
https://auto.bazos.cz/inzerat/157599803/alu-vw-17-5x112-letni-2254517-golftourancaddy.php
ht

https://auto.bazos.cz/inzerat/157535272/skoda-octavia-3-rs-combi.php
https://auto.bazos.cz/inzerat/157533462/tlumice-octavia-3.php
https://auto.bazos.cz/inzerat/157533148/skoda-octavia-14-cng-ambition-g-tec-combi.php
https://auto.bazos.cz/inzerat/157532857/octavia-3-zadni-brzdove-kotouce.php
https://auto.bazos.cz/inzerat/157532840/octavia-3-predni-brzdove-kotouce-a-desticky.php
https://auto.bazos.cz/inzerat/157529195/skoda-octavia-3-16tdi-nahradni-dily.php
https://auto.bazos.cz/inzerat/157528798/zamek-dveri-octavia-3.php
https://auto.bazos.cz/inzerat/157528660/skoda-octavia-3-leve-zrcatko.php
https://auto.bazos.cz/inzerat/157528652/skoda-octavia-3-sedadlo-ridice.php
https://auto.bazos.cz/inzerat/157528315/naraznik-octavia-3-facelift.php
https://auto.bazos.cz/inzerat/157524215/skoda-octavia-iii-pz-dvere-liftback-sedan.php
https://auto.bazos.cz/inzerat/157523449/alu-kola-gemini-r18-original-octavia-3-rs.php
https://auto.bazos.cz/inzerat/157521128/skoda-octavia-3-4x4-20-tdi-110kwwebastota